In [4]:
import json
from pathlib import Path
import tarfile
import tempfile

import nibabel as nib
from nilearn.maskers import NiftiMapsMasker
import numpy as np

In [10]:
p = './NDARINV6G63TU7R_baselineYear1Arm1_ABCD-MPROC-rsfMRI_20180427114950.tgz'
with tarfile.open(p, 'r:gz') as tar:
    tar.extractall()

In [ ]:
p = './NDARINV6G63TU7R_baselineYear1Arm1_ABCD-MPROC-rsfMRI_20180427114950.tgz'
maps_img = nib.load('./difumo64.nii.gz')
save_dir = Path('./')

with tarfile.open(p, 'r:gz') as tar:
    # Extract all members to a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        tar.extractall(path=temp_dir)
        confounds = list(Path(temp_dir).glob('**/*motion.tsv'))[0]
        nii_file = list(Path(temp_dir).glob('**/*.nii'))[0]
        json_file = list(Path(temp_dir).glob('**/*bold.json'))[0]
        name = nii_file.stem

        # Read the JSON file
        with open(json_file) as f:
            json_data = json.load(f)
        t_r = json_data['RepetitionTime']

        img = nib.load(nii_file)

        masker = NiftiMapsMasker(
            maps_img=maps_img,
            standardize="zscore_sample",
            detrend=True,
            t_r=t_r,
            dtype=np.float32,
            verbose=1
        )

        img_masked = masker.fit_transform(img, confounds)
        np.save(save_dir / f'{name}.npy', img_masked)

[NiftiMapsMasker.wrapped] loading regions from None
[NiftiMapsMasker.wrapped] Resampling maps
[NiftiMapsMasker.wrapped] Loading data from Nifti1Image('/tmp/tmp6jjv4q1i/sub-NDARINV6G63TU7R/ses-baselineYear1Arm1/func/sub-NDARINV6G63TU7R_ses-baselineYear1Arm1_task-rest_run-01_bold.nii')
[NiftiMapsMasker.wrapped] Extracting region signals
[NiftiMapsMasker.wrapped] Cleaning extracted signals


In [6]:
data = np.load('./NDARINV6G63TU7R_baselineYear1Arm1_ABCD-MPROC-rsfMRI_20180427114950.npy')
data.shape, data.dtype

((391, 64), dtype('float32'))